In [1]:
from unsloth import FastLanguageModel
from unsloth.chat_templates import get_chat_template
import torch
import pandas as pd
from transformers import AutoTokenizer

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


c:\Users\nikke\GitHub\ai-pentest-report-finetuning-pipeline\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
W0203 09:06:18.442000 22080 Lib\site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
[tensorflow|WARNING]From c:\Users\nikke\GitHub\ai-pentest-report-finetuning-pipeline\.venv\lib\site-packages\tf_keras\src\losses.py:2976: The name tf.losses.sparse_softmax_cross_entropy is deprecated. Please use tf.compat.v1.losses.sparse_softmax_cross_entropy instead.



🦥 Unsloth Zoo will now patch everything to make training faster!
Unsloth: Could not import trl.trainer.alignprop_trainer: Failed to import trl.trainer.alignprop_trainer because of the following error (look up to see its traceback):
Failed to import trl.models.modeling_sd_base because of the following error (look up to see its traceback):
Failed to import diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion because of the following error (look up to see its traceback):
Failed to import diffusers.loaders.ip_adapter because of the following error (look up to see its traceback):
DLL load failed while importing _C: The specified module could not be found.
Unsloth: Could not import trl.trainer.ddpo_trainer: Failed to import trl.trainer.ddpo_trainer because of the following error (look up to see its traceback):
Failed to import trl.models.modeling_sd_base because of the following error (look up to see its traceback):
Failed to import diffusers.pipelines.stable_diffusion.pipeline_sta

In [2]:
MODEL_NAME = "nraesalmi/SET_eval_gemma-3_adapters"
BASE_MODEL = "unsloth/gemma-3-4b-it"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

csv_file = "..\\data\\manual_model_eval_dataset_0.2k.csv"

In [3]:
# Load your CSV
data = pd.read_csv(csv_file)

# Extract instruction and expected output columns
instructions = data["input"].astype(str)
expected_outputs = data["output"].astype(str)

In [4]:
# Load model + tokenizer via Unsloth
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = BASE_MODEL,
    max_seq_length = 2048,     # adjust if needed 8192
    dtype = None,              # auto
    load_in_4bit = True,      # set True if VRAM constrained
)

model.load_adapter(MODEL_NAME)


# Enable inference optimizations
FastLanguageModel.for_inference(model)

print(model.peft_config)

==((====))==  Unsloth 2026.1.4: Fast Gemma3 patching. Transformers: 4.56.2.
   \\   /|    NVIDIA GeForce RTX 3080. Num GPUs = 1. Max memory: 10.0 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.10.0+cu130. CUDA: 8.6. CUDA Toolkit: 13.0. Triton: 3.6.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Gemma3 does not support SDPA - switching to fast eager.
{'default': LoraConfig(task_type='CAUSAL_LM', peft_type=<PeftType.LORA: 'LORA'>, auto_mapping={'base_model_class': 'Gemma3ForConditionalGeneration', 'parent_library': 'transformers.models.gemma3.modeling_gemma3', 'unsloth_fixed': True}, peft_version='0.18.1', base_model_name_or_path='unsloth/gemma-3-4b-it-unsloth-bnb-4bit', revision=None, inference_mode=True, r=8, target_modules='(?:.*?(?:language|text).*?(?:self_attn|attention|attn|mlp|feed_forward|ffn|dense).*?(?:k

In [5]:
import re

def split_sections(text):
    return [s.strip() for s in re.split(r"\n\s*\n", text) if s.strip()]

In [6]:
from sentence_transformers import SentenceTransformer
import numpy as np

sentencemodel = SentenceTransformer("all-MiniLM-L6-v2")

def embed_sections(sections):
    return sentencemodel.encode(sections, normalize_embeddings=True)

In [7]:
def section_similarity(expected_sections, output_sections):
    exp_emb = embed_sections(expected_sections)
    out_emb = embed_sections(output_sections)

    sims = []
    for e in exp_emb:
        sims.append(np.max(out_emb @ e))
    return np.mean(sims)

In [8]:
example_input_1 = """[
  {{
    "SET_name": "encoding.InjectROT47",
    "description": "ROT47-encoded payload injection attempt.",
    "outcome": "Vulnerable",
  }},
  {{
    "SET_name": "latentinjection.ContextCarryover",
    "description": "Latent instruction persistence across turns.",
    "outcome": "Vulnerable",
  }},
  {{
    "recommended_remediations": [
      "Instruction hierarchy enforcement",
      "Prompt boundary enforcement"
    ]
  }}
]"""
example_result_1 = """## Issue Summary:
Analysis revealed recurring vulnerabilities across several evaluation sets. The affected SETs include:
- encoding.InjectROT47
- latentinjection.ContextCarryover

### Recommended Remediations:
To improve system security, it is recommended to enforce Instruction hierarchy enforcement and strengthen Prompt boundary enforcement.
"""

example_input_2 = """[
  {{
    "SET_name": "suffix.UnicodeConfusion",
    "description": "Unicode homoglyph suffix-based injection.",
    "outcome": "Vulnerable",
  }},
  {{
    "SET_name": "packagehallucination.Python",
    "description": "Python package hallucination test.",
    "outcome": "Vulnerable",
  }},
  {{
    "recommended_remediations": [
      "Input validation and normalization",
      "Output validation and consistency checks"
    ]
  }}
]"""

example_result_2 = """## Issue Summary:
The Security Evaluation Tests identified weaknesses in suffix.UnicodeConfusion and packagehallucination.Python.

### Recommended Remediations:
To mitigate the identified risks, it is advised to apply Input validation and normalization and introduce Output validation and consistency checks.
"""

In [ ]:
from transformers import TextStreamer

# Set up tokenizer template
tokenizer = get_chat_template(
    tokenizer,
    chat_template="gemma-3"
)

# Enable faster inference
FastLanguageModel.for_inference(model)

i=0
matches = 0
outputs = []

# Limit to first 10 for testing
# instructions = instructions.head(10)

for instr in instructions:

    expected_output = expected_outputs.iloc[i]
    print(f"\n\n=== Example {i+1} ===")
    print("----------------- Expected Output ----------------\n", expected_output)
    print ("\n------------------- Model Output ------------------\n")

    # Example JSON summarization message with rules
    messages = [
        {"role": "user", "content": [{ "type": "text", "text": f"""You are an AI penetration test summarizing assistant. Summarize the given Security Evaluation Tests (SETs) according to the rules below, strictly based on the provided input.

1. Produce exactly two sentences total.
2. Sentence 1 MUST start with "\n## Issue Summary:\n" and present the weaknesses demonstrated by the SETs and their descriptions.
   - Do NOT introduce impacts, consequences, or behaviors not directly stated or clearly inferable from the descriptions.
3. Sentence 2 MUST start with "\n### Recommended Remediations:\n" and include all recommended_remediations present in the input.
   - The sentence MUST NOT introduce remediations not present in the input, and MUST NOT generalize beyond them.
4. Use neutral, formal, technical language suitable for a security assessment report.
5. Do NOT include explanations, meta-commentary, or generation details.
6. Do NOT claim data access, exfiltration, system compromise, or real-world harm unless explicitly stated in the input.
7. Do NOT introduce speculative attack chains or inferred consequences beyond the SET descriptions.

STRICT OUTPUT TEMPLATE (MANDATORY):
- Sentence 1 MUST start with "## Issue Summary:".
- Sentence 2 MUST start with "### Recommended Remediations:".
- The output MUST contain exactly two sentences and no additional text.

Here are two example Result-Summary pairs
Result 1:
{example_input_1}
Summary 1:
{example_result_1}

Result 2:
{example_input_2}
Summary 2:
{example_result_2}

Here is the penetration test input to summarize:\n{instr}"""}]}
    ]

    # Tokenize with attention mask
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt",
        return_dict=True,
    )

    # Handle single tensor vs dict output
    if isinstance(inputs, dict):
        input_ids = inputs["input_ids"].to("cuda")
        attention_mask = inputs["attention_mask"].to("cuda")
    else:
        input_ids = inputs.to("cuda")
        attention_mask = None

    eos_token_id = tokenizer.convert_tokens_to_ids("<end_of_turn>")

    # Generate output
    model_output = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=128,  # longer outputs for full summary
        temperature=1.0,
        top_p=0.95,
        top_k=64,
        use_cache=True,
        do_sample=False,
        repetition_penalty=1.2,
        eos_token_id=eos_token_id,
    )

    # Compare model output to expected output
    generated_ids = model_output[0][input_ids.shape[-1]:]

    decoded_output = tokenizer.decode(
        generated_ids,
        skip_special_tokens=True
    )

    for end_token in ["<|endoftext|>", "<end>"]:
        decoded_output = decoded_output.replace(end_token, "")

    # Strip extra whitespace
    truncate_output = decoded_output.strip()

    print(truncate_output)

    outputs.append(truncate_output)
    i += 1

    # Calculate similarity score
    score = section_similarity(
        split_sections(expected_output),
        split_sections(truncate_output)
    )

    accuracy = score * 100


    print(f"Accuracy: {accuracy:.2f}%")

    if accuracy >= 70:
        print("Result: ✅ Match")
        matches += 1
    else:
        print("Result: ❌ Mismatch")


print(f"\n\n=== Summary ===\nTotal Matches: {matches}/{len(instructions)}")
pd.DataFrame({"model_output": outputs}).to_csv("../data/ai_outputs.csv", index=False)



=== Example 1 ===
----------------- Expected Output ----------------
 ## Issue Summary:
The findings from the Security Evaluation Tests indicate weaknesses in latentinjection.LatentInjectionFactSnippetEiffelFull, lmrc.Deadnaming, which may impact system robustness.

## Recommended Remediations:
To address the identified vulnerabilities, it is advisable to strengthen Intent detection and output filtering and enforce Prompt boundary enforcement.

------------------- Model Output ------------------



TypeError: string indices must be integers